In [189]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
import sys

In [190]:
####SAVED: EDC-EDML tiepoints from Ruth et al. 2007 and Severi et al. 2007, with EDC96 depths converted to EDC99 depths

# sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/EDML-EDC_Severi2007.xls', sheet_name=0, skiprows=45, usecols=[0,1], names=['EDML', 'EDC_both']) 
# sv_links = sv_links[1:-4]

# edc_convert = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/EDC96-EDC99.xlsx', sheet_name=0, skiprows=1, usecols=[0,1], names=['EDC96', 'EDC99'])

# comment_add = 'Includes EDML-EDC links from Ruth et al. 2007 and Severi et al. 2007, original file with interpolated EDC96 depths to EDC99'

# interp_values = np.interp(
#     sv_links['EDC_both'],
#     edc_convert['EDC96'],
#     edc_convert['EDC99']
# )

# sv_links['EDC'] = np.where(sv_links['EDML'] >= 1439.06, sv_links['EDC_both'], interp_values)

# sv_links = sv_links[['EDML', 'EDC']]

# sv_links.loc[sv_links['EDML'] == 1435.08, 'EDC'] = 777.682

####SAVED: EDC-WDC tiepoint Sigl email interpolation

#sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/WD-TALDICE-Byrd-EDML-EDC_Sigl_01142025.xlsx', sheet_name=3, skiprows=30, usecols=[0,1], names=['WDC', 'EDC96'])

# edc_convert = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/EDC96-EDC99.xlsx', sheet_name=0, skiprows=1, usecols=[0,1], names=['EDC96', 'EDC99'])

# sv_links['EDC'] = np.interp(
#     sv_links['EDC96'],
#     edc_convert['EDC96'],
#     edc_convert['EDC99']
# )

# comment_add = 'Includes WDC-EDC links from Sigl email on Jan 14 2025 (forwarded by Christo) with interpolated EDC96 depths to EDC99'
# sv_links = sv_links[['WDC', 'EDC']]

#####SAVED: DF-WDC tiepoints from Ikumi email on Nov 8, 2025 with interpolation of DF2 to DF1 links

#comment_add = 'Includes DF-WDC tiepoints from Ikumi email on Nov 8, 2025. Includes interpolation of DF2 to DF1 links.'

# sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/DF_WDC_SPC_tiepoints_forQuinn.xlsx', sheet_name=0, skiprows=34, usecols=[0,3,5], names=['DF1', 'DF2', 'WDC'])

# sv_links.dropna(subset=['WDC'], inplace=True)

# calibrate = sv_links[['DF1', 'DF2']].dropna()

# sv_links['DF'] = np.where(~sv_links['DF1'].isna(), sv_links['DF1'], np.interp(sv_links['DF2'], calibrate['DF2'], calibrate['DF1']))

# sv_links = sv_links[['DF', 'WDC']]

# sv_links = sv_links[(sv_links['DF'] < 979)]

# sv_links

In [191]:
comment_add = 'Includes WDC-TALDICE tiepoints from Sigl email on Jan 14 2025 (forwarded by Christo)'

sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/WD-TALDICE-Byrd-EDML-EDC_Sigl_01142025.xlsx', sheet_name=1, skiprows=0, usecols=[2,3], names=['WDC', 'TALDICE'])

sv_links

,WDC,TALDICE
0,154.80500,72.01600
1,161.43309,74.72389
2,168.86983,77.96997
3,182.31137,83.47874
4,186.19864,85.97287
...,...,...
517,3362.47460,1259.50340
518,3362.96100,1259.91820
519,3367.32100,1263.54820
520,3370.74630,1267.60680


In [192]:
backups=False
folder='AA_14Cols'
error=10
tolerance = 0.10  # meters

if backups == True:
    # add backup of BICC

    source_folder = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}'
    backup_root = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}/Backups'

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_folder = os.path.join(backup_root, f'BICC_Backup_{timestamp}')

    # Make sure the backup root exists
    os.makedirs(backup_root, exist_ok=True)

    # Copy the folder
    shutil.copytree(source_folder, backup_folder)

    print(f"Backup completed: {backup_folder}")
else:
    print("Backups not enabled.")


Backups not enabled.


In [193]:
#load cores list from params
params = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [194]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['age_unc'] = error
    sv_synchros[pair]['comment'] = np.nan  # Initialize age column with NaN

    sv_comments[pair] = [comment_add]

#now load existing synchros
os.chdir(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}')
existing_synchros = {}
existing_comments = {}

for pair in valid_pairs:
    folder_path = pair
    file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")

    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):

        comments = []
        with open(file_path, 'r') as f: # store existing comments to be re-added
            for line in f:
                if line.startswith('#'):
                    comments.append(line.rstrip('\n'))  # keeps tabs intact  # store comment lines without newline
        existing_comments[pair] = comments

        try:
            synchros = pd.read_csv(file_path, sep='\t', comment='#')
        except Exception as e:
            print(f"Error loading {pair}: {e}")
            print(f'File Path: {file_path}')
            sys.exit()

        existing_synchros[pair] = synchros
        print(f"Loaded {pair}: {len(synchros)} rows")
    else:
        print(f"Skipped {pair} — file not found")

Loaded WDC-TALDICE: 569 rows


In [195]:
for pair in valid_pairs:

    if pair in existing_synchros:
        
        overlaps = 0
        new_ties = 0
        error_overlaps = 0

        existing = existing_synchros[pair].copy(deep=True)
        new = sv_synchros[pair].copy(deep=True)

        total_existing = len(existing)

        # Start with originals (priority)
        combined = existing.copy(deep=True)

        # Append only existing tiepoints that are NOT close to any new tiepoint
        for idx, row in new.iterrows():

            mask_either = ( #check for either column within tolerance.
                (abs(existing.iloc[:, 0] - row.iloc[0]) <= tolerance) |
                (abs(existing.iloc[:, 1] - row.iloc[1]) <= tolerance)
            )
            mask_both = ( #check for both columns to spit out error if needed
                (abs(existing.iloc[:, 0] - row.iloc[0]) <= tolerance) &
                (abs(existing.iloc[:, 1] - row.iloc[1]) <= tolerance)
            )

            if not mask_either.any():
                combined = pd.concat([combined, row.to_frame().T], ignore_index=True)
                new_ties+=1
            
            if mask_both.any():
                overlaps += 1

            if mask_either.any() and not mask_both.any():
                print(f"Warning: Partial overlap detected for pair {pair} at new tiepoint {row.values}. One column is within {tolerance} of an existing tiepoint, while other is not. The tiepoint WAS kept for comparison purposes.")
                combined = pd.concat([combined, row.to_frame().T], ignore_index=True)
                error_overlaps += 1
                new_ties+=1

        #duplicate check
        before = len(combined)
        combined = combined.drop_duplicates(ignore_index=True)
        after = len(combined)

        if after < before:
            print(f"Warning: {before - after} duplicate tiepoint(s) found and removed in pair {pair}.")

        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        # --- Gather comments ---
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            if new_ties > 0:
                for line in new_comments:
                    f.write(f"#{line}\n")
            # Add original comments
            for line in existing_comments.get(pair, []):
                f.write(f"#{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)
        print(f"Saved merged synchro for {pair} with rows to {file_path}\nThere were {total_existing} existing rows, {overlaps} full overlaps ({error_overlaps} error overlaps), and {new_ties} new ties.")

    else:
        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        combined = sv_synchros[pair].copy(deep=True) # svensson links load
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            for line in new_comments:
                f.write(f"#{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)

        print(f"No existing synchros for {pair}, saving new synchro to {file_path}")


Saved merged synchro for WDC-TALDICE with rows to WDC-TALDICE/iceice_synchro_horizons.txt
There were 569 existing rows, 454 full overlaps (49 error overlaps), and 68 new ties.
